In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import xlsxwriter
pd.options.display.float_format = '{:,.2f}'.format

## READING FILES

In [2]:
excel_name = 'Dispersión-20220420.xlsx'
date_respuesta = '2022-04-20'
date_respuesta = pd.to_datetime(date_respuesta)

In [3]:
disp = pd.read_csv(r'G:\My Drive\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\04. APRIL\SWAP 2022-04-20.csv'
                     ,dtype={'bank_account_number':object,'report_id':object})
rs1 = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\2022\04. ABRIL\Respuesta-Swap 2022-04-20.csv',
                  skiprows=11,dtype={'Descripcion':object,'Referencia Numerica':object,'Rastreo CEP':object})
cep1 = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\2022\04. ABRIL\CEP 2022-04-20.txt'
                ,names=['Fecha','Clave_rastreo','Banco_emisor','Banco_receptor','Cuenta','Monto']
                   ,dtype={'Fecha':object,'Clave_rastreo':object,'Banco_emisor':object,'Banco_receptor':object,'Cuenta':object})
bin1 = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\Catalogo_Bancos.csv'
                   ,dtype={'BIN':object,'Clave de la institución':object,'Nombre de la institución':object})


In [4]:
rs1.head()

,Referencia Numerica,Rastreo CEP,Nombre,Descripcion,Cuenta,Banco,Importe,Comision,Estatus,Mensaje
0,3100001,2022042040014 BET0000460061610,OLIVER IVAN RODRIGUEZ DE LOS REYES,222Y6TB6H,'058597000022362906,BANREGIO,0.01,2.15,paid,NaN
1,3100002,2022042040014 BET0000460062150,EMMANUEL ESCARTIN MORENO,229Y8Y4YD,'002441903436087128,BANAMEX,0.95,2.15,paid,NaN
2,3100003,2022042040014 BET0000460062520,JIMMY ROGER ORTEGA LANZ,22NQMVYPR,'127041001295191597,AZTECA,"2,432.01",2.15,paid,NaN
3,3100004,2022042040014 BET0000460062980,YOEL YANIK ORTIZ AVILES,22QL9PXBY,'137180104403070656,BANCOPPEL,0.01,2.15,paid,NaN
4,3100005,2022042040014 BET0000460063660,STEVEN ORLANDO DAVILA SAMACON,22WY2NNFJ,'127320013866638619,AZTECA,"2,856.39",2.15,paid,NaN


In [5]:
rs1.shape

(79203, 10)

# BIN

In [6]:
bin1.head()

,BIN,Clave de la institución,Nombre de la institución
0,138,40138,ABC CAPITAL
1,102,40102,ACCENDO BANCO
2,133,40133,ACTINVER
3,062,40062,AFIRME
4,638,90638,AKALA


In [7]:
bin1.shape

(89, 3)

# ARCHIVO CEPS

In [8]:
cep1['Linea_CEP'] = (cep1['Fecha']+','
                     +cep1['Clave_rastreo']+','
                     +cep1['Banco_emisor']+','
                     +cep1['Banco_receptor']+','
                     +cep1['Cuenta']+','
                     +cep1['Monto'].astype(str))

In [9]:
cep1.head()

,Fecha,Clave_rastreo,Banco_emisor,Banco_receptor,Cuenta,Monto,Linea_CEP
0,20-04-2022,2022042040014 BET0000460061610,40014,40058,058597000022362906,0.01,"20-04-2022,2022042040014 BET0000460061610,4001..."
1,20-04-2022,2022042040014 BET0000460062150,40014,40002,002441903436087128,0.95,"20-04-2022,2022042040014 BET0000460062150,4001..."
2,20-04-2022,2022042040014 BET0000460062520,40014,40127,127041001295191597,"2,432.01","20-04-2022,2022042040014 BET0000460062520,4001..."
3,20-04-2022,2022042040014 BET0000460062980,40014,40137,137180104403070656,0.01,"20-04-2022,2022042040014 BET0000460062980,4001..."
4,20-04-2022,2022042040014 BET0000460063660,40014,40127,127320013866638619,"2,856.39","20-04-2022,2022042040014 BET0000460063660,4001..."


In [10]:
cep1.shape

(79203, 7)

# ARCHIVO RESPUESTA

In [11]:
rs1['Descripcion'] = rs1['Descripcion'].str.strip()

In [12]:
rs1['Date_respuesta'] = date_respuesta
rs1['Metodo de dispersion'] = 'SWAP'
rs1.rename(columns ={'Descripcion':'report_id'}, inplace=True)
rs1['Rastreo CEP'].fillna('SIN CEP',inplace=True)
rs1.loc[rs1['Estatus'].str.contains('paid'), "Estatus"] = 'PAID'
rs1.loc[rs1['Estatus'].str.contains('failed'), "Estatus"] = 'ERROR'
rs1.head()

,Referencia Numerica,Rastreo CEP,Nombre,report_id,Cuenta,Banco,Importe,Comision,Estatus,Mensaje,Date_respuesta,Metodo de dispersion
0,3100001,2022042040014 BET0000460061610,OLIVER IVAN RODRIGUEZ DE LOS REYES,222Y6TB6H,'058597000022362906,BANREGIO,0.01,2.15,PAID,NaN,2022-04-20,SWAP
1,3100002,2022042040014 BET0000460062150,EMMANUEL ESCARTIN MORENO,229Y8Y4YD,'002441903436087128,BANAMEX,0.95,2.15,PAID,NaN,2022-04-20,SWAP
2,3100003,2022042040014 BET0000460062520,JIMMY ROGER ORTEGA LANZ,22NQMVYPR,'127041001295191597,AZTECA,"2,432.01",2.15,PAID,NaN,2022-04-20,SWAP
3,3100004,2022042040014 BET0000460062980,YOEL YANIK ORTIZ AVILES,22QL9PXBY,'137180104403070656,BANCOPPEL,0.01,2.15,PAID,NaN,2022-04-20,SWAP
4,3100005,2022042040014 BET0000460063660,STEVEN ORLANDO DAVILA SAMACON,22WY2NNFJ,'127320013866638619,AZTECA,"2,856.39",2.15,PAID,NaN,2022-04-20,SWAP


In [13]:
rs1.shape

(79203, 12)

In [14]:
rs1['Rastreo CEP'].str[:4].unique()

array(['2022', '#N/D', '0', 'CLIP', '1804'], dtype=object)

In [15]:
rs1.loc[rs1['Rastreo CEP'].str.contains("'085"), "Linea_rastreo"] = rs1['Rastreo CEP'].str[1:]
rs1.loc[~rs1['Rastreo CEP'].str.contains("'085"), "Linea_rastreo"] = rs1['Rastreo CEP']


In [16]:
rs1.loc[rs1['Rastreo CEP'].str[:4] == "'085", "Banco_emisor"] = 'BANAMEX'
rs1.loc[rs1['Rastreo CEP'].str[:4] == 'CLIP', "Banco_emisor"] = 'STP'
rs1.loc[rs1['Rastreo CEP'].str[:4] == '2021', "Banco_emisor"] = 'SANTANDER'
rs1.loc[rs1['Rastreo CEP'].str[:4] == '2022', "Banco_emisor"] = 'SANTANDER'
rs1.loc[rs1['Rastreo CEP'].str[:4] == '#N/D', "Banco_emisor"] = 'SANTANDER'
rs1.loc[rs1['Rastreo CEP'].str[:3] == '178', "Banco_emisor"] = 'BANAMEX'
rs1.loc[rs1['Rastreo CEP'].str[:2] == '17', "Banco_emisor"] = 'BANAMEX'
rs1.loc[rs1['Rastreo CEP'].str[:4] == '1.78', "Banco_emisor"] = 'BANAMEX'
rs1.loc[rs1['Rastreo CEP'].str[:4] == '1.79', "Banco_emisor"] = 'BANAMEX'
rs1.loc[rs1['Rastreo CEP'].str[:4] == '0', "Banco_emisor"] = 'SANTANDER - REFERENCIA NUMERICA'
rs1.loc[rs1['Rastreo CEP'].str[:4] == '#¡REF!', "Banco_emisor"] = 'CEP PENDIENTE DE ENVIO'


In [17]:
rs1.loc[rs1['Banco_emisor'] == 'BANAMEX', "Emisor"] = '40002'
rs1.loc[rs1['Banco_emisor'] == 'STP', "Emisor"] = '90646'
rs1.loc[rs1['Banco_emisor'] == 'SANTANDER', "Emisor"] = '40014' 


In [18]:
rs1['Cuenta'].str[0:1].unique()

array(["'"], dtype=object)

In [19]:
rs1['BIN'] = rs1['Cuenta'].str[1:4]
rs1['CLABE'] = rs1['Cuenta'].str[1:]


In [20]:
rs1.head()

,Referencia Numerica,Rastreo CEP,Nombre,report_id,Cuenta,Banco,Importe,Comision,Estatus,Mensaje,Date_respuesta,Metodo de dispersion,Linea_rastreo,Banco_emisor,Emisor,BIN,CLABE
0,3100001,2022042040014 BET0000460061610,OLIVER IVAN RODRIGUEZ DE LOS REYES,222Y6TB6H,'058597000022362906,BANREGIO,0.01,2.15,PAID,NaN,2022-04-20,SWAP,2022042040014 BET0000460061610,SANTANDER,40014,058,058597000022362906
1,3100002,2022042040014 BET0000460062150,EMMANUEL ESCARTIN MORENO,229Y8Y4YD,'002441903436087128,BANAMEX,0.95,2.15,PAID,NaN,2022-04-20,SWAP,2022042040014 BET0000460062150,SANTANDER,40014,002,002441903436087128
2,3100003,2022042040014 BET0000460062520,JIMMY ROGER ORTEGA LANZ,22NQMVYPR,'127041001295191597,AZTECA,"2,432.01",2.15,PAID,NaN,2022-04-20,SWAP,2022042040014 BET0000460062520,SANTANDER,40014,127,127041001295191597
3,3100004,2022042040014 BET0000460062980,YOEL YANIK ORTIZ AVILES,22QL9PXBY,'137180104403070656,BANCOPPEL,0.01,2.15,PAID,NaN,2022-04-20,SWAP,2022042040014 BET0000460062980,SANTANDER,40014,137,137180104403070656
4,3100005,2022042040014 BET0000460063660,STEVEN ORLANDO DAVILA SAMACON,22WY2NNFJ,'127320013866638619,AZTECA,"2,856.39",2.15,PAID,NaN,2022-04-20,SWAP,2022042040014 BET0000460063660,SANTANDER,40014,127,127320013866638619


In [21]:
rs1.shape

(79203, 17)

In [22]:
rs2 = rs1.merge(bin1[['BIN','Clave de la institución','Nombre de la institución']] ,on='BIN', how='left')
rs2['LINEA_CEP'] = (rs2['Date_respuesta'].astype(str).replace(' ', '',regex=True)+','
                    +rs2['Linea_rastreo'].astype(str)+','
                    +rs2['Emisor'].astype(str).replace(' ', '',regex=True)+','
                    +rs2['Clave de la institución'].astype(str).replace(' ', '',regex=True)+','
                    +rs2['CLABE'].astype(str).replace(' ', '',regex=True)+','
                    +rs2['Importe'].astype(str).replace(' ', '',regex=True))
rs2.head()

,Referencia Numerica,Rastreo CEP,Nombre,report_id,Cuenta,Banco,Importe,Comision,Estatus,Mensaje,Date_respuesta,Metodo de dispersion,Linea_rastreo,Banco_emisor,Emisor,BIN,CLABE,Clave de la institución,Nombre de la institución,LINEA_CEP
0,3100001,2022042040014 BET0000460061610,OLIVER IVAN RODRIGUEZ DE LOS REYES,222Y6TB6H,'058597000022362906,BANREGIO,0.01,2.15,PAID,NaN,2022-04-20,SWAP,2022042040014 BET0000460061610,SANTANDER,40014,058,058597000022362906,40058,BANREGIO,"2022-04-20,2022042040014 BET0000460061610,4001..."
1,3100002,2022042040014 BET0000460062150,EMMANUEL ESCARTIN MORENO,229Y8Y4YD,'002441903436087128,BANAMEX,0.95,2.15,PAID,NaN,2022-04-20,SWAP,2022042040014 BET0000460062150,SANTANDER,40014,002,002441903436087128,40002,BANAMEX,"2022-04-20,2022042040014 BET0000460062150,4001..."
2,3100003,2022042040014 BET0000460062520,JIMMY ROGER ORTEGA LANZ,22NQMVYPR,'127041001295191597,AZTECA,"2,432.01",2.15,PAID,NaN,2022-04-20,SWAP,2022042040014 BET0000460062520,SANTANDER,40014,127,127041001295191597,40127,AZTECA,"2022-04-20,2022042040014 BET0000460062520,4001..."
3,3100004,2022042040014 BET0000460062980,YOEL YANIK ORTIZ AVILES,22QL9PXBY,'137180104403070656,BANCOPPEL,0.01,2.15,PAID,NaN,2022-04-20,SWAP,2022042040014 BET0000460062980,SANTANDER,40014,137,137180104403070656,40137,BANCOPPEL,"2022-04-20,2022042040014 BET0000460062980,4001..."
4,3100005,2022042040014 BET0000460063660,STEVEN ORLANDO DAVILA SAMACON,22WY2NNFJ,'127320013866638619,AZTECA,"2,856.39",2.15,PAID,NaN,2022-04-20,SWAP,2022042040014 BET0000460063660,SANTANDER,40014,127,127320013866638619,40127,AZTECA,"2022-04-20,2022042040014 BET0000460063660,4001..."


In [23]:
rs2.shape

(79203, 20)

# ARCHIVO DISPERSION

In [24]:
disp['report_id'] = disp['report_id'].str.strip()
disp['report_date_local'] = pd.to_datetime(disp['report_date_local'])

In [25]:
disp.loc[disp['report_date_local'] == date_respuesta, 'tipo'] = 'Pago del día'
disp.loc[disp['report_date_local'] != date_respuesta, 'tipo'] = 'Reprogramación'
disp.drop(['bank_name','bank_account_number','status','swap','payment_date_local'],axis=1,inplace=True)
disp.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,bank_account_owner_name,batch_created_date,disbursement_method,tipo
0,00008884-ca93-4758-8aa7-b10f57145850,Dra Abryl,maah25@hotmail.com,VN76SZYZ3,2022-04-20,"1,149.89",Abryl Mariana Avitia Herrera,2022-04-20 08:03:18,SWAP,Pago del día
1,0000b65a-c2a2-4993-8f4c-05e5b7cbd7c0,Ruvalcaba Peleteros S.A. de C.V.,ruvalcabapeleteros@hotmail.com,VJDSB5MVH,2022-04-20,"1,041.41",Ruvalcaba Peleteros SA de CV,2022-04-20 08:03:18,SWAP,Pago del día
2,0000d758-84b4-4cb0-a165-4f0d3c44864d,MAHARISHI PANTJALI SA DE CV,maharishiclip@gmail.com,78KN95CWP,2022-04-20,"26,631.71",MAHARISHI PANTJALI S.A. DE C.V.,2022-04-20 08:03:18,SWAP,Pago del día
3,0000e953-6b2c-44d0-a798-9dac1abf24ea,Super Kandy,yanellysegovia18@gmail.com,JZM3R2ZN5,2022-04-20,755.76,Ana Yanelly Puente Gutierrez,2022-04-20 08:03:18,SWAP,Pago del día
4,00016126-0052-402d-ba8a-95e5e1a1c673,Seguridad Industrial OM,jocabetolan@gmail.com,ZTJJXKZRG,2022-04-20,861.46,CYNTHIA JOCABET OLAN RODRIGUEZ,2022-04-20 08:03:18,SWAP,Pago del día


In [26]:
disp_2 = disp.merge(rs2[['report_id','Estatus','Nombre de la institución','CLABE','Banco_emisor','LINEA_CEP',
                         'Referencia Numerica']] ,on='report_id', how='left')
disp_2.sort_values(by=['Estatus'],ascending=True,inplace=True)
disp_2.sort_values(by=['Estatus'],ascending=True,inplace=True)
disp_2['Estatus'].fillna('not_found',inplace=True)
disp_2['Nombre de la institución'].fillna('not_found',inplace=True)
disp_2['CLABE'].fillna('not_found',inplace=True)
disp_2['Banco_emisor'].fillna('not_found',inplace=True)
disp_2['LINEA_CEP'].fillna('not_found',inplace=True)
disp_2['Referencia Numerica'].fillna('not_found',inplace=True)
disp_2.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,bank_account_owner_name,batch_created_date,disbursement_method,tipo,Estatus,Nombre de la institución,CLABE,Banco_emisor,LINEA_CEP,Referencia Numerica
32168,6864f02b-abf0-4ea1-a428-9516eae8ed4e,Dloto Huaracheria,dlotohuaracheria@gmail.com,QHS843TPX,2022-04-20,253.93,Bernice Torres Maldonado,2022-04-20 08:08:32,SWAP,Pago del día,ERROR,BBVA BANCOMER,012180015693652700,SANTANDER - REFERENCIA NUMERICA,"2022-04-20,0,nan,40012,012180015693652700,253.93",3254897
3451,0ae18b46-9993-4e8e-8665-0b57038a93ec,Los Volcanes,losvolcanesfloresmontalvo24@gmail.com,T94LLK5VG,2022-04-19,"1,359.73",Josefina Flores Montalvo,2022-04-20 08:04:07,SWAP,Reprogramación,ERROR,COMPARTAMOS,130670001533951365,SANTANDER - REFERENCIA NUMERICA,"2022-04-20,0,nan,40130,130670001533951365,1359.73",3262397
3450,0ae18b46-9993-4e8e-8665-0b57038a93ec,Los Volcanes,losvolcanesfloresmontalvo24@gmail.com,WLHPLKF7M,2022-04-18,982.20,Josefina Flores Montalvo,2022-04-20 08:04:07,SWAP,Reprogramación,ERROR,COMPARTAMOS,130670001533951365,SANTANDER - REFERENCIA NUMERICA,"2022-04-20,0,nan,40130,130670001533951365,982.2",3268591
75771,f4da94ee-31a8-498f-940f-3abe1eee9eaa,BOHO,iltze.cz@gmail.com,7PH6YJ482,2022-04-20,9.00,MARCELA SILVIA HERNANDEZ PEREZ,2022-04-20 08:14:27,SWAP,Pago del día,ERROR,BANCOPPEL,137180104195876197,SANTANDER - REFERENCIA NUMERICA,"2022-04-20,0,nan,40137,137180104195876197,9.0",3215283
8327,1aa8c2f4-da52-4cb3-af13-6d55c9aeb061,Tortillas de Harina de Sonora,alopezdavila@hotmail.com,QXRL2PCKR,2022-04-20,189.73,Sergio Adrian Lopez Davila,2022-04-20 08:05:08,SWAP,Pago del día,ERROR,BANAMEX,002073560179460525,STP,"2022-04-20,CLIP3256120,90646,40002,00207356017...",229175013


In [27]:
disp_2.loc[disp_2['LINEA_CEP'].str.contains('#N/D'), 'LINEA_CEP'] = 'COMPROBANTE MISMO BANCO SANTANDER'
disp_2['LINEA_CEP'] = (np.where(disp_2['Banco_emisor'] == 'SANTANDER - REFERENCIA NUMERICA',
                                disp_2['Referencia Numerica'], disp_2['LINEA_CEP']))
disp_2.loc[disp_2['Estatus'] == 'ERROR', 'LINEA_CEP'] = ''
disp_2.loc[disp_2['Estatus'] == 'ERROR', 'Banco_emisor'] = ''
disp_2.loc[disp_2['LINEA_CEP'] == 'CEP PENDIENTE DE ENVIO', 'Banco_emisor'] = ''

In [28]:
for col in disp_2.columns: 
    print(col) 

merchant_id
merchant_name
user_email
report_id
report_date_local
amount
bank_account_owner_name
batch_created_date
disbursement_method
tipo
Estatus
Nombre de la institución
CLABE
Banco_emisor
LINEA_CEP
Referencia Numerica


In [29]:
disp_final = disp_2[['merchant_id','merchant_name','user_email','report_id','report_date_local','amount','tipo'
                    ,'Nombre de la institución','bank_account_owner_name','CLABE','Estatus','LINEA_CEP','Banco_emisor']]
disp_final.rename(columns ={'Nombre de la institución':'bank','Estatus':'status','amount':'total_due'}, inplace=True)
disp_final.sort_values(by=['status'], ascending=True, inplace=True)
disp_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_8080\2121322040.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disp_final.rename(columns ={'Nombre de la institución':'bank','Estatus':'status','amount':'total_due'}, inplace=True)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_8080\2121322040.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disp_final.sort_values(by=['status'], ascending=True, inplace=True)


,merchant_id,merchant_name,user_email,report_id,report_date_local,total_due,tipo,bank,bank_account_owner_name,CLABE,status,LINEA_CEP,Banco_emisor
32168,6864f02b-abf0-4ea1-a428-9516eae8ed4e,Dloto Huaracheria,dlotohuaracheria@gmail.com,QHS843TPX,2022-04-20,253.93,Pago del día,BBVA BANCOMER,Bernice Torres Maldonado,012180015693652700,ERROR,,
55344,b39f031f-aa84-4618-b281-1e93df87f258,Miscelanea Betos,luisalbertomontielr@gmail.com,SGLBT3XZL,2022-04-20,67.08,Pago del día,COMPARTAMOS,Luis Alberto Montiel Rivera,130670001543852292,ERROR,,
33671,6d6cf701-6872-4ee6-9596-897ff053f375,cafetería Vizcaya,roberto_0099@icloud.com,RJCKYFJWY,2022-04-19,120.74,Reprogramación,COMPARTAMOS,Roberto Carlos Serrano Hernandez,130062001289863483,ERROR,,
63257,ccec151d-cd87-43ba-b196-9acc10b5bda9,GO&CI,aj.enrique_2017@hotmail.com,6DPJKK8KM,2022-04-15,939.08,Reprogramación,BAJIO,Elvia Arminda Cid Ramirez,030426900005682504,ERROR,,
25960,539f06ac-ea3a-4352-9e69-6efdcbdf7209,MyD store,npi_linea@hotmail.com,86VNVBFG3,2022-04-20,"2,515.38",Pago del día,BANAMEX,Tomasa Laguna Perez,002073560177947024,ERROR,,


In [30]:
disp_final.shape

(79203, 13)

In [31]:
duplicates_disp_final = disp_final[disp_final.duplicated(['report_id'],keep=False)]
duplicates_disp_final.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,total_due,tipo,bank,bank_account_owner_name,CLABE,status,LINEA_CEP,Banco_emisor


In [32]:
duplicates_disp_final.shape

(0, 13)

# ERROR

In [33]:
error = disp_final[(disp_final['status'] != 'PAID')]
error.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,total_due,tipo,bank,bank_account_owner_name,CLABE,status,LINEA_CEP,Banco_emisor
32168,6864f02b-abf0-4ea1-a428-9516eae8ed4e,Dloto Huaracheria,dlotohuaracheria@gmail.com,QHS843TPX,2022-04-20,253.93,Pago del día,BBVA BANCOMER,Bernice Torres Maldonado,012180015693652700,ERROR,,
55344,b39f031f-aa84-4618-b281-1e93df87f258,Miscelanea Betos,luisalbertomontielr@gmail.com,SGLBT3XZL,2022-04-20,67.08,Pago del día,COMPARTAMOS,Luis Alberto Montiel Rivera,130670001543852292,ERROR,,
33671,6d6cf701-6872-4ee6-9596-897ff053f375,cafetería Vizcaya,roberto_0099@icloud.com,RJCKYFJWY,2022-04-19,120.74,Reprogramación,COMPARTAMOS,Roberto Carlos Serrano Hernandez,130062001289863483,ERROR,,
63257,ccec151d-cd87-43ba-b196-9acc10b5bda9,GO&CI,aj.enrique_2017@hotmail.com,6DPJKK8KM,2022-04-15,939.08,Reprogramación,BAJIO,Elvia Arminda Cid Ramirez,030426900005682504,ERROR,,
25960,539f06ac-ea3a-4352-9e69-6efdcbdf7209,MyD store,npi_linea@hotmail.com,86VNVBFG3,2022-04-20,"2,515.38",Pago del día,BANAMEX,Tomasa Laguna Perez,002073560177947024,ERROR,,


In [34]:
error.shape

(309, 13)

In [35]:
error.drop(['bank_account_owner_name','CLABE','status','LINEA_CEP','Banco_emisor'],axis=1,inplace=True)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_8080\704642815.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error.drop(['bank_account_owner_name','CLABE','status','LINEA_CEP','Banco_emisor'],axis=1,inplace=True)


In [36]:
error.loc[error['bank'] == 'SANTANDER', 'Check'] = 'SPEI DEVUELTO MISMO BANCO'
error.loc[error['bank'] != 'SANTANDER', 'Check'] = 'SPEI DEVUELTO'

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_8080\2883469485.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error.loc[error['bank'] == 'SANTANDER', 'Check'] = 'SPEI DEVUELTO MISMO BANCO'


In [37]:
error_final = error.merge(rs2[['report_id','Mensaje']] ,on='report_id', how='left')
error_final['Mensaje'].fillna('NOT FOUND',inplace=True)
error_final.head()


,merchant_id,merchant_name,user_email,report_id,report_date_local,total_due,tipo,bank,Check,Mensaje
0,6864f02b-abf0-4ea1-a428-9516eae8ed4e,Dloto Huaracheria,dlotohuaracheria@gmail.com,QHS843TPX,2022-04-20,253.93,Pago del día,BBVA BANCOMER,SPEI DEVUELTO,Excede el limite de saldo autorizado de la cue...
1,b39f031f-aa84-4618-b281-1e93df87f258,Miscelanea Betos,luisalbertomontielr@gmail.com,SGLBT3XZL,2022-04-20,67.08,Pago del día,COMPARTAMOS,SPEI DEVUELTO,Tipo de operacion erronea
2,6d6cf701-6872-4ee6-9596-897ff053f375,cafetería Vizcaya,roberto_0099@icloud.com,RJCKYFJWY,2022-04-19,120.74,Reprogramación,COMPARTAMOS,SPEI DEVUELTO,Tipo de operacion erronea
3,ccec151d-cd87-43ba-b196-9acc10b5bda9,GO&CI,aj.enrique_2017@hotmail.com,6DPJKK8KM,2022-04-15,939.08,Reprogramación,BAJIO,SPEI DEVUELTO,Cuenta cancelada
4,539f06ac-ea3a-4352-9e69-6efdcbdf7209,MyD store,npi_linea@hotmail.com,86VNVBFG3,2022-04-20,"2,515.38",Pago del día,BANAMEX,SPEI DEVUELTO,Excede el límite de abonos permitidos en el me...


In [38]:
error_final.shape

(309, 10)

# ARCHIVOS FINALES

### ARCHIVO DISPERSION

In [39]:
writer = pd.ExcelWriter(excel_name, engine='xlsxwriter', engine_kwargs={'options': {'encoding':'utf-8'}})
disp_final.to_excel(writer, sheet_name='Swap del día',index=False)
error_final.to_excel(writer, sheet_name='ERROR',index=False)
writer.close()

### ARCHIVO RESPUESTA

In [40]:
rs2.head()

,Referencia Numerica,Rastreo CEP,Nombre,report_id,Cuenta,Banco,Importe,Comision,Estatus,Mensaje,Date_respuesta,Metodo de dispersion,Linea_rastreo,Banco_emisor,Emisor,BIN,CLABE,Clave de la institución,Nombre de la institución,LINEA_CEP
0,3100001,2022042040014 BET0000460061610,OLIVER IVAN RODRIGUEZ DE LOS REYES,222Y6TB6H,'058597000022362906,BANREGIO,0.01,2.15,PAID,NaN,2022-04-20,SWAP,2022042040014 BET0000460061610,SANTANDER,40014,058,058597000022362906,40058,BANREGIO,"2022-04-20,2022042040014 BET0000460061610,4001..."
1,3100002,2022042040014 BET0000460062150,EMMANUEL ESCARTIN MORENO,229Y8Y4YD,'002441903436087128,BANAMEX,0.95,2.15,PAID,NaN,2022-04-20,SWAP,2022042040014 BET0000460062150,SANTANDER,40014,002,002441903436087128,40002,BANAMEX,"2022-04-20,2022042040014 BET0000460062150,4001..."
2,3100003,2022042040014 BET0000460062520,JIMMY ROGER ORTEGA LANZ,22NQMVYPR,'127041001295191597,AZTECA,"2,432.01",2.15,PAID,NaN,2022-04-20,SWAP,2022042040014 BET0000460062520,SANTANDER,40014,127,127041001295191597,40127,AZTECA,"2022-04-20,2022042040014 BET0000460062520,4001..."
3,3100004,2022042040014 BET0000460062980,YOEL YANIK ORTIZ AVILES,22QL9PXBY,'137180104403070656,BANCOPPEL,0.01,2.15,PAID,NaN,2022-04-20,SWAP,2022042040014 BET0000460062980,SANTANDER,40014,137,137180104403070656,40137,BANCOPPEL,"2022-04-20,2022042040014 BET0000460062980,4001..."
4,3100005,2022042040014 BET0000460063660,STEVEN ORLANDO DAVILA SAMACON,22WY2NNFJ,'127320013866638619,AZTECA,"2,856.39",2.15,PAID,NaN,2022-04-20,SWAP,2022042040014 BET0000460063660,SANTANDER,40014,127,127320013866638619,40127,AZTECA,"2022-04-20,2022042040014 BET0000460063660,4001..."


In [41]:
rs_final = rs2[['Referencia Numerica','Rastreo CEP','Nombre','report_id','Cuenta','Banco','Importe','Comision','Estatus'
                ,'Mensaje','Date_respuesta','Metodo de dispersion']]
rs_final.sort_values(by=['Estatus'], ascending=True, inplace=True)
rs_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_8080\4035833657.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rs_final.sort_values(by=['Estatus'], ascending=True, inplace=True)


,Referencia Numerica,Rastreo CEP,Nombre,report_id,Cuenta,Banco,Importe,Comision,Estatus,Mensaje,Date_respuesta,Metodo de dispersion
7570,3208344,0,SANDRA KARINA CORONA TORRES,55JGP8BZL,'012261015275014012,BBVA BANCOMER,95.82,2.15,ERROR,Excede el limite de saldo autorizado de la cue...,2022-04-20,SWAP
15896,3231023,0,ALMA ROSA HERNANDEZ MOLINA,FKK9HFV76,'044680256010111588,SCOTIABANK,287.47,2.15,ERROR,Falta informacion mandatoria para completar el...,2022-04-20,SWAP
5489,3204188,0,JOSE GUADALUPE RIVERA BARRERA,3KRM8SJYT,'012822015352732494,BBVA BANCOMER,83.36,2.15,ERROR,Cuenta bloqueada,2022-04-20,SWAP
58327,3257672,0,ROBERTO CARLOS SERRANO HERNANDEZ,RJCKYFJWY,'130062001289863483,COMPARTAMOS,120.74,2.15,ERROR,Tipo de operacion erronea,2022-04-20,SWAP
17710,3216547,0,JOSE CARMEN CAPORAL HUERTA,85ZDRZLQV,'021240040633967827,HSBC,132.44,2.15,ERROR,Cuenta bloqueada,2022-04-20,SWAP


In [42]:
rs_final.shape

(79203, 12)

In [43]:
rs_final.to_csv(r'G:\My Drive\3.-Settlement\Respuestas Swap\2022\04. Abril 2022\Respuesta 2022-04-20.csv',index=False)